In [14]:
import pandas as pd
from datetime import datetime

In [15]:
# Load your price data
price_data = pd.read_csv("Nat_Gas Total Profit.csv")
price_data['Dates'] = pd.to_datetime(price_data['Dates'], format="%m/%d/%y")

In [16]:
# ---- Step 1: Helper function to get price for a given date ----
def estimate_price(date):
    date = pd.to_datetime(date)
    closest = price_data.iloc[(price_data['Dates'] - date).abs().argsort()[:1]]
    return float(closest['Prices'].values[0])

In [18]:
# ---- Step 2: Main pricing function ----
def price_storage_contract(injection_dates, withdrawal_dates,
                           injection_rate, withdrawal_rate,
                           max_storage, storage_cost,
                           inj_fee=0, wdr_fee=0):
    """
    Calculates total profit/value of a natural gas storage contract.
    """
    total_injected = min(injection_rate * len(injection_dates), max_storage)
    total_withdrawn = min(withdrawal_rate * len(withdrawal_dates), total_injected)

    # Get prices
    buy_prices = [estimate_price(d) for d in injection_dates]
    sell_prices = [estimate_price(d) for d in withdrawal_dates]

    # Total revenue from sales
    revenue = sum(p * withdrawal_rate for p in sell_prices)

    # Total cost of buying gas
    cost = sum(p * injection_rate for p in buy_prices)

    # Fees and storage cost
    total_fees = (inj_fee * total_injected) + (wdr_fee * total_withdrawn)
    total_costs = cost + storage_cost + total_fees

    # Net profit
    profit = revenue - total_costs
    return {
        "Total Revenue": revenue,
        "Total Cost": total_costs,
        "Profit": profit
    }

In [19]:
# ---- Step 3: Example test ----
result = price_storage_contract(
    injection_dates=["2020-10-31", "2020-11-30"],
    withdrawal_dates=["2021-01-31", "2021-02-28"],
    injection_rate=250000,
    withdrawal_rate=250000,
    max_storage=500000,
    storage_cost=100000,
    inj_fee=0.1,     # Optional
    wdr_fee=0.1      # Optional
)

In [21]:
print()
print(f"The value of the contract is: ${result}")


The value of the contract is: ${'Total Revenue': 5450000.0, 'Total Cost': 5300000.0, 'Profit': 150000.0}
